# Building the LSTM Model

In [ ]:
import torch
from datetime import datetime
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split, Subset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import torch.optim as optim
import torch.nn as nn
import sys
import os
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from datetime import datetime
import threading
import webbrowser
from optuna.visualization import plot_optimization_history
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from datetime import datetime
import time
import subprocess
import webbrowser
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from helper.helper import float_to_time, time_to_float, float_time_range, float_time_minus
import random
import tqdm
# Set seed for repeatability
def seed_everything(seed):
    np.random.seed(seed) # np random seed
    random.seed(seed) # py random seed
seed_everything(seed=1024)
import torch
print('torch version: ',torch.__version__)
print('Cuda available: ',torch.cuda.is_available())
print('Running on ',torch.cuda.get_device_name(torch.cuda.current_device()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
X_lstm_tensor = torch.load('../../data/X_lstm_train.pt',weights_only=False)
t_lstm_tensor = torch.load('../../data/t_lstm_train.pt',weights_only=False)

In [ ]:
nan_count = torch.isnan(X_lstm_tensor).sum().item()
print(f"Number of NaN values in X: {nan_count}")
nan_count = torch.isnan(t_lstm_tensor).sum().item()
print(f"Number of NaN values in t: {nan_count}")


In [ ]:
X_lstm_tensor.shape, t_lstm_tensor.shape

In [20]:
lstm_hr_df = pl.read_csv('../../data/lstm_hr_train_clean.csv')
bg_columns = ['bg', 'bg+1:00']
bg_max = lstm_hr_df['bg'].max()
bg_min = lstm_hr_df['bg'].min()
print(f"Max BG: {bg_max}, Min BG: {bg_min}")
bg_data = lstm_hr_df.select(bg_columns).to_numpy()

bg_scaler = MinMaxScaler()
bg_scaler.fit(lstm_hr_df['bg'].to_numpy().reshape(-1, 1)) 


Max BG: 27.8, Min BG: 2.2


MinMaxScaler()

In [ ]:
dataset = TensorDataset(X_lstm_tensor, t_lstm_tensor)

patient_columns = X_lstm_tensor[:, :, 2:10]
patient_columns
class_labels = patient_columns.argmax(dim=2)[:, 0]


train_indices, temp_indices = train_test_split(
    range(len(dataset)),
    test_size=0.5,  # 50% for val+test
    stratify=class_labels
)

val_indices, test_indices = train_test_split(
    temp_indices,
    test_size=0.5,  # Half of temp for test, other half for val
    stratify=class_labels[temp_indices]
)

# Create subsets
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def create_dataloaders(X_lstm_tensor, t_tensor, batch_size, val_split=0.4):
    dataset_size = len(X_lstm_tensor)
    indices = list(range(dataset_size))
    split = int(np.floor(val_split * dataset_size))
    
    train_indices, val_indices = indices[split:], indices[:split]
    
    train_loader = DataLoader(
        list(zip(X_lstm_tensor[train_indices], t_tensor[train_indices])),
        batch_size=batch_size,
        shuffle=True
    )
    
    val_loader = DataLoader(
        list(zip(X_lstm_tensor[val_indices], t_tensor[val_indices])),
        batch_size=batch_size,
        shuffle=False
    )
    
    return train_loader, val_loader

In [ ]:
from collections import Counter
import numpy as np

# Function to count class distribution
def check_proportions(dataset, description):
    # Extract the one-hot encoded columns
    one_hot_columns = X_lstm_tensor[dataset.indices, :, 2:11]
    
    # Convert one-hot encoding to class labels
    class_labels = one_hot_columns.argmax(dim=2)[:, 0].numpy()  # Use the first timestep's label
    
    # Count occurrences
    class_counts = Counter(class_labels)
    total_samples = sum(class_counts.values())
    
    print(f"{description} Class Proportions:")
    for cls, count in sorted(class_counts.items()):
        print(f"Class {cls}: {count} samples ({count / total_samples:.2%})")
    print("-" * 50)

# Check proportions for each split
check_proportions(train_dataset, "Train")
check_proportions(val_dataset, "Validation")
check_proportions(test_dataset, "Test")


In [ ]:
for X, t in train_loader:
    print('X.shape:', X.shape, 't.shape:', t.shape)
    break
for X, t in val_loader:
    print('X.shape:', X.shape, 't.shape:', t.shape)
    break
for X, t in test_loader:
    print('X.shape:', X.shape, 't.shape:', t.shape)
    break

In [17]:
import torch.nn as nn
import json

class LSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMRegressor, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        output, (hn, cn) = self.lstm(x)
        last_hidden_state = output[:, -1, :]
        prediction = self.fc(last_hidden_state)
        return prediction

INPUT_SIZE = X_lstm_tensor.shape[-1]

class TrialProgress:
    def __init__(self, study_name):
        self.study_name = study_name
        self.results_dir = "optuna_results"
        os.makedirs(self.results_dir, exist_ok=True)
        self.results_file = os.path.join(self.results_dir, f"{study_name}_results.json")
        self.current_best = float('inf')
        self.results = {
            "study_name": study_name,
            "trials": [],
            "best_trial": None,
            "best_value": float('inf')
        }
        
    def update(self, trial, value):
        trial_info = {
            "number": trial.number,
            "params": trial.params,
            "value": value,
            "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        self.results["trials"].append(trial_info)
        if value < self.results["best_value"]:
            self.results["best_value"] = value
            self.results["best_trial"] = trial_info
            self.current_best = value
        
        # Save results to file
        with open(self.results_file, 'w') as f:
            json.dump(self.results, f, indent=2)

In [19]:
def objective(trial):
    trial_number = trial.number
    
    # Hyperparameters to optimize
    hidden_size = trial.suggest_int('hidden_size', 32, 256)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    
    print(f"\nTrial {trial_number} started")
    print(f"Parameters: {trial.params}")
    
    model = LSTMRegressor(
        input_size=INPUT_SIZE,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)
    
    train_loader, val_loader = create_dataloaders(X_lstm_tensor, t_lstm_tensor, batch_size)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    best_val_loss = float('inf')
    patience = 5
    patience_counter = 0
    max_epochs = 50
    
    for epoch in range(max_epochs):
        # Training loop
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = torch.sqrt(criterion(predictions, y_batch))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val_batch, y_val_batch in val_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
                val_predictions = model(X_val_batch)
                loss = torch.sqrt(criterion(val_predictions, y_val_batch))
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        
        # Print progress
        print(f"\rEpoch {epoch:3d}/{max_epochs:3d} | "
              f"Train RMSE: {avg_train_loss:.4f} | Val RMSE: {avg_val_loss:.4f} | "
              f"Best overall: {progress_tracker.current_best:.4f}", end="")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered at epoch {epoch}")
                break
        
        trial.report(avg_val_loss, epoch)
        if trial.should_prune():
            print(f"\nTrial {trial_number} pruned.")
            raise optuna.exceptions.TrialPruned()
    
    # Update progress tracker
    progress_tracker.update(trial, best_val_loss)
    
    print(f"\nTrial {trial_number} finished with best val_loss: {best_val_loss:.4f}")
    return best_val_loss

# Create study
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
study_name = f"lstm_optimization_{timestamp}"

# Initialize progress tracker
progress_tracker = TrialProgress(study_name)

# Create study
study = optuna.create_study(
    study_name=study_name,
    direction='minimize',
    pruner=optuna.pruners.MedianPruner()
)

print("Starting optimization...")
print(f"Study name: {study_name}")
print(f"Results will be saved to: {progress_tracker.results_file}")
print("\nTo monitor progress:")
print(f"1. Check the terminal output for real-time updates")
print(f"2. View the JSON results file: {progress_tracker.results_file}")

try:
    study.optimize(objective, n_trials=100, timeout=43200)  # 12 hours timeout
except KeyboardInterrupt:
    print("\nOptimization interrupted by user.")

# Final results
print("\nOptimization completed!")
print("\nBest trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-11-23 02:35:27,434] A new study created in memory with name: lstm_optimization_20241123_023527


Starting optimization...
Study name: lstm_optimization_20241123_023527
Results will be saved to: optuna_results/lstm_optimization_20241123_023527_results.json

To monitor progress:
1. Check the terminal output for real-time updates
2. View the JSON results file: optuna_results/lstm_optimization_20241123_023527_results.json

Trial 0 started
Parameters: {'hidden_size': 204, 'num_layers': 1, 'dropout': 0.44409212356202415, 'batch_size': 128, 'learning_rate': 0.005910714624583147}
Epoch  27/ 50 | Train RMSE: 0.0738 | Val RMSE: 0.0744 | Best overall: inf
Early stopping triggered at epoch 27

Trial 0 finished with best val_loss: 0.0659


[I 2024-11-23 02:41:22,732] Trial 0 finished with value: 0.06594620736319227 and parameters: {'hidden_size': 204, 'num_layers': 1, 'dropout': 0.44409212356202415, 'batch_size': 128, 'learning_rate': 0.005910714624583147}. Best is trial 0 with value: 0.06594620736319227.



Trial 1 started
Parameters: {'hidden_size': 220, 'num_layers': 3, 'dropout': 0.47460816687392127, 'batch_size': 16, 'learning_rate': 0.0019550937147834214}
Epoch   5/ 50 | Train RMSE: 0.0876 | Val RMSE: 0.0796 | Best overall: 0.0659
Early stopping triggered at epoch 5

Trial 1 finished with best val_loss: 0.0757


[I 2024-11-23 02:50:27,015] Trial 1 finished with value: 0.07572863223757967 and parameters: {'hidden_size': 220, 'num_layers': 3, 'dropout': 0.47460816687392127, 'batch_size': 16, 'learning_rate': 0.0019550937147834214}. Best is trial 0 with value: 0.06594620736319227.


...

Epoch   3/ 50 | Train RMSE: 0.0732 | Val RMSE: 0.0746 | Best overall: 0.0494
Trial 97 pruned.

Trial 98 started
Parameters: {'hidden_size': 119, 'num_layers': 3, 'dropout': 0.3920271579243134, 'batch_size': 64, 'learning_rate': 0.0029279545083165516}


[I 2024-11-23 13:09:55,989] Trial 98 pruned. 


Epoch   0/ 50 | Train RMSE: 0.0839 | Val RMSE: 0.0799 | Best overall: 0.0494
Trial 98 pruned.

Trial 99 started
Parameters: {'hidden_size': 100, 'num_layers': 1, 'dropout': 0.430917550166707, 'batch_size': 32, 'learning_rate': 0.00250238184515355}


[I 2024-11-23 13:10:10,018] Trial 99 pruned. 


Epoch   0/ 50 | Train RMSE: 0.0796 | Val RMSE: 0.0772 | Best overall: 0.0494
Trial 99 pruned.

Optimization completed!

Best trial:
  Value:  0.04938529467653017
  Params: 
    hidden_size: 102
    num_layers: 1
    dropout: 0.3376473541285027
    batch_size: 64
    learning_rate: 0.0026930312968810326


In [ ]:
import torch.optim as optim
import torch.nn as nn

INPUT_SIZE = X_lstm_tensor.shape[-1]
HIDDEN_SIZE = 102
BATCH_SIZE = 64
EPOCHS = 50
LEARTNING_RATE =  0.0026930312968810326

LSTMReg = LSTMRegressor(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE)
LSTMReg.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(LSTMReg.parameters(), lr=LEARTNING_RATE)

for epoch in range(EPOCHS):
    LSTMReg.train()
    train_loss = 0.0  # For accumulating training loss
    
    for X_batch, t_batch in train_loader:
        X_batch, t_batch = X_batch.to(device), t_batch.to(device)
        
        optimizer.zero_grad()  # Clear gradients
        predictions = LSTMReg(X_batch)  # Forward pass
        
        loss = torch.sqrt(criterion(predictions, t_batch))
        loss.backward()  # Backward pass
        optimizer.step()  # Update model parameters
        
        train_loss += loss.item()  # Accumulate batch loss

    # Compute average train loss (MSE)
    avg_train_loss = train_loss / len(train_loader)

    # Validation loop
    LSTMReg.eval()
    val_loss = 0.0  # For accumulating validation loss
    with torch.no_grad():  # Disable gradient computation for validation
        for X_val_batch, t_val_batch in val_loader:
            X_val_batch, t_val_batch = X_val_batch.to(device), t_val_batch.to(device)
            val_predictions = LSTMReg(X_val_batch)  # Forward pass
            
            loss = torch.sqrt(criterion(val_predictions, t_val_batch))
            val_loss += loss.item()  # Accumulate batch loss

    # Compute average validation loss (MSE)
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch [{epoch + 1}/{EPOCHS}], Train MSE: {avg_train_loss:.4f}, Val MSE: {avg_val_loss:.4f}")


In [ ]:
LSTMReg.eval()

squared_error_total = 0.0  
num_samples = 0 

with torch.no_grad():
    for X_test_batch, t_test_batch in test_loader:
        X_test_batch, t_test_batch = X_test_batch.to(device), t_test_batch.to(device)
        
        test_predictions = LSTMReg(X_test_batch)
        
        t_test_batch = t_test_batch.view(-1, 1)
        test_predictions = test_predictions.view(-1, 1)

        combined_data = np.hstack([t_test_batch.cpu().numpy(), test_predictions.cpu().numpy()])
        
        t_test_batch_inverse, test_predictions_inverse = bg_scaler.inverse_transform(combined_data).T
        t_test_batch_inverse = torch.from_numpy(t_test_batch_inverse)
        test_predictions_inverse = torch.from_numpy(test_predictions_inverse)


        squared_error = (test_predictions_inverse - t_test_batch_inverse) ** 2

        squared_error_total += squared_error.sum().item() 
        num_samples += len(t_test_batch_inverse)  


average_squared_error = squared_error_total / num_samples


test_rmse = np.sqrt(average_squared_error)

print(f"Test RMSE: {test_rmse:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

LSTMReg.eval()

def calculate_rmse(predictions, targets):
    return torch.sqrt(((predictions - targets) ** 2).mean())

test_rmse = 0.0
num_batches = 0

all_targets = []
all_predictions = []
last_bg = []

with torch.no_grad():  
    for X_test_batch, t_test_batch in test_loader:
        X_test_batch, t_test_batch = X_test_batch.to(device), t_test_batch.to(device)
        
        test_predictions = LSTMReg(X_test_batch)
        
        # Flatten both targets and predictions for proper shape handling
        t_test_batch = t_test_batch.view(-1, 1)
        test_predictions = test_predictions.view(-1, 1)
        
        # Collect last bg values (assuming 13th index is where bg is located)
        last_bg_batch = X_test_batch[:,-1,13].cpu().numpy()
        
        # Inverse transform the last bg, targets, and predictions
        last_bg_inverse = bg_scaler.inverse_transform(last_bg_batch.reshape(-1, 1))
        t_test_batch_inverse = bg_scaler.inverse_transform(t_test_batch.cpu().numpy())
        test_predictions_inverse = bg_scaler.inverse_transform(test_predictions.cpu().numpy())
        
        # Convert back to tensors
        t_test_batch_inverse = torch.from_numpy(t_test_batch_inverse)
        test_predictions_inverse = torch.from_numpy(test_predictions_inverse)

        # Append all values to lists
        all_targets.append(t_test_batch_inverse.cpu().numpy())
        all_predictions.append(test_predictions_inverse.cpu().numpy())
        last_bg.append(last_bg_inverse)

        # Calculate RMSE for the current batch and update total
        test_rmse += calculate_rmse(test_predictions_inverse, t_test_batch_inverse).item()
        num_batches += 1

# Concatenate the results for all batches
all_targets = np.concatenate(all_targets)
all_predictions = np.concatenate(all_predictions)
all_last_bg = np.concatenate(last_bg)

# Calculate the average RMSE over all batches
test_rmse /= num_batches
print(f"Test RMSE: {test_rmse:.4f}")

# Sort the data based on targets
sorted_indices = np.argsort(all_targets)  
sorted_targets = all_targets[sorted_indices] 
sorted_predictions = all_predictions[sorted_indices]  
sorted_last_bg = all_last_bg[sorted_indices]

# Check lengths for debugging
print(len(sorted_targets), len(sorted_predictions), len(sorted_last_bg))


In [ ]:
start,end = 0, len(sorted_targets)
plt.figure(figsize=(20, 12))
plt.plot(sorted(all_predictions), label='Predictions', color='red', linestyle='--')
plt.plot(sorted(all_targets), label='True Values', color='blue')
plt.plot(sorted(all_last_bg), label='Last BG', color='green')

plt.xlabel('Index (sorted by Target Value)')
plt.ylabel('Value')
plt.title('True vs Predicted Values (Sorted by Target Value)')
plt.legend()
plt.show()

In [ ]:
X_

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20, 10))
plt.plot(np.arange(len(all_targets)), all_targets, '.', alpha=0.5)
plt.plot(np.arange(len(all_predictions)), all_predictions, '.', alpha=0.5)
plt.xlabel('Index')
plt.ylabel('Target Value')
plt.title('Plot of all_targets')
plt.show()


In [ ]:
X_lstm_tensor = torch.load('../../data/X_lstm_test.pt',weights_only=False)

submission_ids = X_lstm_tensor[:, 0, -1]
X_lstm_tensor = X_lstm_tensor[:,:,:-1].to(device)

all_ids = []
for ids in submission_ids:
    all_ids.append([float_id_converter(ids),ids])

In [ ]:
pandas_ids = pd.DataFrame(all_ids, columns=['id', 'float_id'])
pandas_ids.describe()
pandas_ids

In [ ]:
def float_id_converter(float_id):
    # Extract the integer part and fractional part
    int_part = int(float_id)
    frac_part = np.round((float_id - int_part) * 1e5)
    
    formatted_int_part = f'{int_part:02}'  
    formatted_frac_part = f'{int(frac_part)}'
    
    return f'p{formatted_int_part}_{formatted_frac_part}'

float_id_converter(3.0001)



In [ ]:
LSTMReg.eval()


with torch.no_grad():  
    predictions = LSTMReg(X_lstm_tensor).cpu().numpy()

# Now, prepare the output in the desired format
output_lines = []
for i in range(len(submission_ids)):
    float_id = submission_ids[i].item()  # Convert to scalar if it's a tensor
    id_str = float_id_converter(float_id)  # Convert to the desired string format
    
    # Assuming that bg+1:00 is the first predicted value
    bg_value = predictions[i, 0]  # You can adjust this if your model outputs multiple columns
    bg_value = bg_scaler.inverse_transform([[bg_value]])  # Inverse transform
    bg_value = bg_value[0, 0]  # Extract the scalar value
    print(bg_value)
    # Format the output
    output_lines.append(f"{id_str},{bg_value}")  # Format bg_value to 2 decimal places

# Save or print the output
output = "\n".join(output_lines)

# Optionally, write to a CSV file
with open("submission_output.csv", "w") as f:
    f.write("id,bg+1:00\n")
    f.write(output)
